In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1096,1392,655,728
1,./images\N100.xml,132,305,301,353
2,./images\N101.xml,21,141,127,165
3,./images\N102.xml,161,318,206,251
4,./images\N103.xml,807,1071,659,738


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images\\N1.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images\\N1.jpeg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N140.jpeg',
 './images\\N141.jpeg',
 './images\\N142.j

In [8]:
#### Verify image and Output

In [9]:
file_path = image_path[0]
file_path

'./images\\N1.jpeg'

In [10]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
#  1096	1392	655	728
cv2.rectangle(img,(1096,655),(1392,728),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
#### Data Preprocessing

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [14]:
labels = df.iloc[:,1:].values

In [15]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    
    #preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/ 255.0 #normalization
    
    #normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w, xmax/w
    nymin,nymax = ymin/h, ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) #normalized output

    #---------------------append-------------
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [16]:
x = np.array(data,dtype = np.float32)
y = np.array(output,dtype = np.float32)

In [17]:
x.shape,y.shape

((197, 224, 224, 3), (197, 4))

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.8, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape 

((157, 224, 224, 3), (40, 224, 224, 3), (157, 4), (40, 4))

In [19]:
#### Deep Learning Model

In [20]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [21]:
inception_resnet = InceptionV3(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
# headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(64,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [31]:
# compile model
model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 25,079,908 (95.67 MB)

 Trainable params: 3,277,124 (12.50 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [23]:
#### Training Object Detection Model

In [32]:
from tensorflow.keras.callbacks import TensorBoard
tfb = TensorBoard('object_detection')

In [33]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=10,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 610ms/step - accuracy: 0.9845 - loss: 1.4353e-04 - val_accuracy: 0.5500 - val_loss: 0.0347
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 318ms/step - accuracy: 0.9841 - loss: 1.0943e-04 - val_accuracy: 0.5250 - val_loss: 0.0346
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 328ms/step - accuracy: 0.9855 - loss: 1.0283e-04 - val_accuracy: 0.5500 - val_loss: 0.0347
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 316ms/step - accuracy: 0.9773 - loss: 9.2358e-05 - val_accuracy: 0.5250 - val_loss: 0.0347
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 359ms/step - accuracy: 0.9851 - loss: 7.4976e-05 - val_accuracy: 0.5250 - val_loss: 0.0348
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 353ms/step - accuracy: 0.9980 - loss: 5.1416e-05 - val_accuracy: 0.5250 - val_loss: 0.0349
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 327ms/step - accuracy: 0.9973 - loss: 4.7577e-05 - val_accuracy: 0.5250 - val_loss: 0.0348
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 362ms/step - accuracy: 0.9881 - lo

In [36]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=10)

Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 493ms/step - accuracy: 0.9989 - loss: 4.2932e-05 - val_accuracy: 0.5500 - val_loss: 0.0349
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 330ms/step - accuracy: 0.9972 - loss: 4.1152e-05 - val_accuracy: 0.5250 - val_loss: 0.0350
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 340ms/step - accuracy: 0.9886 - loss: 4.1206e-05 - val_accuracy: 0.5250 - val_loss: 0.0347
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 7s 401ms/step - accuracy: 0.9920 - loss: 3.3202e-05 - val_accuracy: 0.5250 - val_loss: 0.0351
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 354ms/step - accuracy: 0.9750 - loss: 4.4873e-05 - val_accuracy: 0.5250 - val_loss: 0.0348
Epoch 16/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 345ms/step - accuracy: 0.9927 - loss: 4.3225e-05 - val_accuracy: 0.5250 - val_loss: 0.0351
Epoch 17/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 328ms/step - accuracy: 0.9642 - loss: 3.2658e-05 - val_accuracy: 0.5500 - val_loss: 0.0349
Epoch 18/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 348ms/step - accur

In [27]:
model.save('./newModel/object_detection.h5')

In [37]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy: {:2.2f}%'.format(test_acc*100))

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 307ms/step - accuracy: 0.5437 - loss: 0.0323
Test accuracy: 55.00%
